In [ ]:
!pip install pyngrok 

In [ ]:
import os, io, time, asyncio
import torch, numpy as np
from queue import Queue
from threading import Thread
from typing import List
import torch.nn.functional as F
from fastapi import FastAPI
from fastapi import UploadFile, File
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio
nest_asyncio.apply()
from transformers import AutoTokenizer, AutoModel


QWEN_MODEL_NAME = "Qwen/Qwen3-Embedding-8B"
QWEN_DEVICE = "cuda:0"
PORT = 7005         
MAX_BATCH = 24
MAX_WAIT = 0.01
NGROK_AUTH_TOKEN = "35Q4PzgSja5h0Vo3eJHH1Lf2sdn_5VjWqbgPk3NSsyXLeCuQ2"  


print("📥 Loading Qwen tokenizer...")
qwen_tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_NAME, trust_remote_code=True)
print("⚡ Loading Qwen3-Embedding-8B...")
qwen_model = AutoModel.from_pretrained(
    QWEN_MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map={"": QWEN_DEVICE}
).eval()
print("✅ Qwen embedding model ready.\n")


qwen_txt_q = Queue()

def run_worker(q, fn):
    def w():
        while True:
            batch, cbs = [], []

            x, cb = q.get()
            batch.append(x); cbs.append(cb)
            t0 = time.time()

            while len(batch) < MAX_BATCH and (time.time() - t0) < MAX_WAIT:
                try:
                    x, cb = q.get_nowait()
                    batch.append(x); cbs.append(cb)
                except:
                    break

            out = fn(batch)
            for o, cb in zip(out, cbs):
                cb(o)

    Thread(target=w, daemon=True).start()


run_worker(
    qwen_txt_q,
    lambda B: F.normalize(
        qwen_model(
            **qwen_tokenizer(
                B,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
            ).to(QWEN_DEVICE)
        ).last_hidden_state[:, 0, :], 
        dim=-1
    ).detach().cpu().numpy()
)


app = FastAPI()

# ==============================================================================
# REQUEST/RESPONSE MODELS
# ==============================================================================
class TextReq(BaseModel):
    text: str

class BatchTextReq(BaseModel):
    texts: List[str]

class EmbeddingResponse(BaseModel):
    model: str
    embedding: List[float]
    dimension: int

class BatchEmbeddingResponse(BaseModel):
    model: str
    embeddings: List[List[float]]
    dimension: int
    count: int


# ==============================================================================
# SINGLE REQUEST ENDPOINT
# ==============================================================================
@app.post("/embedding/qwen/text")
async def qwen_text(req: TextReq):
    """
    Single text embedding for IC model (Qwen).
    """
    loop = asyncio.get_running_loop()
    fut = loop.create_future()

    qwen_txt_q.put((req.text, lambda r: loop.call_soon_threadsafe(fut.set_result, r)))
    v = await fut
    v = v.astype("float32")

    return {
        "model": "qwen-text",
        "embedding": v.tolist(),
        "dimension": len(v)
    }


# ==============================================================================
# BATCH REQUEST ENDPOINT
# ==============================================================================
@app.post("/embedding/qwen/text/batch")
async def qwen_text_batch(req: BatchTextReq):
    """
    Batch text embedding for IC model (Qwen).
    Processes multiple texts in one GPU forward pass.
    Perfect for query augmentation (Q0, Q1, Q2).
    """
    texts = req.texts
    if not texts:
        return {"model": "qwen-text", "embeddings": [], "dimension": 0, "count": 0}
    
    # Process all texts in batch
    loop = asyncio.get_running_loop()
    fut = loop.create_future()
    qwen_txt_q.put((texts, lambda r: loop.call_soon_threadsafe(fut.set_result, r)))
    
    v = await fut  # Shape: (batch_size, embedding_dim)
    v = v.astype("float32")
    
    # Convert to list of embeddings
    embeddings = [emb.tolist() for emb in v]
    
    return {
        "model": "qwen-text",
        "embeddings": embeddings,
        "dimension": len(embeddings[0]) if embeddings else 0,
        "count": len(embeddings)
    }


# ==============================================================================
# HEALTH CHECK
# ==============================================================================
@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "model": QWEN_MODEL_NAME,
        "device": QWEN_DEVICE,
        "batch_support": True,
        "max_batch_size": MAX_BATCH
    }


if __name__ == "__main__":
    print("=" * 80)
    print("🚀 IC Model Server (Qwen3-Embedding-8B)")
    print("=" * 80)
    print(f"📦 Model: {QWEN_MODEL_NAME}")
    print(f"🎯 Device: {QWEN_DEVICE}")
    print(f"⚡ Batch processing enabled (max: {MAX_BATCH})")
    print(f"🌐 Port: {PORT}")
    print("=" * 80)
    
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(PORT).public_url
        print(f"🔗 NGROK URL: {public_url}")
        print("=" * 80)

    uvicorn.run(app, host="0.0.0.0", port=PORT)
